# Implementation of the `scaredyFish` model in Julia
### First step
Let's first define a spatial landscape and set parameter values

In [252]:
## ---
## init
## ---
using Distributions, Distances, IterTools, BenchmarkTools


## -
## Define my landscape
## -

## number of rows and cols
nrows = 2
ncols = 4

## landscape is a binary matrix, where 1s are foraging patches (white tiles)
landscape = rand( Bernoulli( 0.5 ), nrows, ncols )

## indices for white tiles
whiteTiles = getindex.( findall( y -> y == 1, landscape ), [ 1 2 ] )


## -
## Define a bunch of parameters
## -

## number of agents
agents = 15

## time steps
time = 20

## walk step, this is the SD of the normal distribution from which walk step lengths are drawn
walk = 0.05

## run step, this is the SD of the normal distribution from which run step lengths are drawn
run = 0.2

## strength of drift back toward white tile if you're in a black tile
drift = 0.2

## excitation strength
strengthExcitation = 0.5

## inhibition strength
strengthInhibition = 0.01

## sensory length scale radius
sensory = 2.5

## decision threshold
threshold = 1

## trigger time for false startle
trigger = 5;

### Second step
Define a function that will loop through `times` and `agents` and do all the calculations

In [253]:
## ---
## Function to loop through time steps
## ---
function scaredyFish( )
    
    ## -
    ## Prep simulation, define some empty arrays to fill in
    ## -

    ## empty 3D array to populate ( time x number of variables x agents )
    ## my columns are | 1 time | 2 agent id | 3 x | 4 y | 5 state | 6 in a white tile |
    r = zeros( time, 6, agents );

    ## populate first row of each matrix
    for agent in 1 : size( r, 3 )

        ## put a random number in the first row
        r[ 1, 1, agent ] = 1
        r[ 1, 2, agent ] = agent
        
        ## sample a white tile
        sampledTile = whiteTiles[ sample( 1 : size( whiteTiles )[ 1 ], 1 ), : ]
        
        ## put a random number in the first row
        r[ 1, 3, agent ] = sampledTile[ 1 ] + Float64.( rand( Uniform( -0.49, 0.49 ) ) )
        r[ 1, 4, agent ] = sampledTile[ 2 ] + Float64.( rand( Uniform( -0.49, 0.49 ) ) )
        
        ## fill in state and location
        r[ 1, 5, agent ] = 0
        r[ 1, 6, agent ] = 1

    end
    
    
    ## -
    ## Start workhorse
    ## -
    
    ## for t in ___time___
    for t in 2 : time

        ## for a in ___agents____
        for a in 1 : agents
            
            ## add info to table
            r[ t, 1, a ] = t

            ## add info to table
            r[ t, 2, a ] = a
            
            ## -
            ## first we need to know if we're in a white tile or black tile, and where the nearest white tile is
            ## -

            ## grab my current x and y before updating
            currX = r[ t - 1, 3, a ]
            currY = r[ t - 1, 4, a ]

            ## round them to find the nearest tile
            currPatchX = round.( Int, currX )
            currPatchY = round.( Int, currY )

            ## check that it is actually a tile, and you are not out of bounds
            if ( 0.5 < currPatchX < ( 0.5 + size( landscape, 1 ) ) ) && ( 0.5 < currPatchY < ( 0.5 + size( landscape, 2 ) ) )

                ## note that I am in bounds
                inBounds = 1

                ## if this is a white tile, we are already done
                if landscape[ currPatchX, currPatchY ] == 1

                    ## make this my home patch
                    homePatchX = currPatchX
                    homePatchY = currPatchY

                    ## record status in output ( 1 if I'm in a white tile, 0 if not )
                    r[ t, 6, a ] = 1

                ## otherwise, find the nearest white tile
                else

                    ## distances to white tiles
                    toWhiteTiles = colwise( Euclidean(), [ currX, currY ], transpose( whiteTiles ) )

                    ## which min
                    homePatchX = whiteTiles[ argmin( toWhiteTiles ), 1 ]
                    homePatchY = whiteTiles[ argmin( toWhiteTiles ), 2 ]

                    ## record status in output
                    r[ t, 6, a ] = 0

                end

            ## if I'm out of bounds...
            else

                ## note it
                inBounds = 0

                ## distances to white tiles
                toWhiteTiles = colwise( Euclidean(), [ currX, currY ], transpose( whiteTiles ) )

                ## which min
                homePatchX = whiteTiles[ argmin( toWhiteTiles ), 1 ]
                homePatchY = whiteTiles[ argmin( toWhiteTiles ), 2 ]

                ## record status in output
                r[ t, 6, a ] = 0

            end


            ## -
            ## Here starts the step updating section
            ## -

            ## if feeding and in bounds
            if r[ t - 1, 5, a ] == 0 && inBounds == 1

                ## random walk steps
                r[ t, 3, a ] = r[ t - 1, 3, a ] + rand( Normal( 0, walk ) )
                r[ t, 4, a ] = r[ t - 1, 4, a ] + rand( Normal( 0, walk ) )

            end

            ## if feeding and out of bounds
            if r[ t - 1, 5, a ] == 0 && inBounds == 0

                ## bias my walk in this direction
                xBias = ( homePatchX - r[ t - 1, 3, a ] )
                yBias = ( homePatchY - r[ t - 1, 4, a ] )

                ## random walk steps
                r[ t, 3, a ] = r[ t - 1, 3, a ] + rand( Normal( xBias * drift, walk ) )
                r[ t, 4, a ] = r[ t - 1, 4, a ] + rand( Normal( yBias * drift, walk ) )

            end

            ## if fleeing
            if r[ t - 1, 5, a ] == 1 && r[ t - 2, 5, a ] == 0

                ## random walk steps
                r[ t, 3, a ] = r[ t - 1, 3, a ] + rand( Normal( 0, run ) )
                r[ t, 4, a ] = r[ t - 1, 4, a ] + rand( Normal( 0, run ) )

            end

        end

        
        ## -
        ## Here starts the information updating section
        ## -

        ## extract current coordinates
        C = hcat( r[ t, 3, : ], r[ t, 4, : ] )

        ## calculate distances between agents
        D = pairwise( Euclidean(), C; dims = 1 )

        ## threshold this matrix
        D[ D .> sensory ] .= 0

        ## convert to 1 / D ^ 2
        D[ D .> 0 ] = 1 ./ D[ D .> 0 ] .^ 2
        
        ##
        ## cycle through agents and calculate excitation and inhibition
        ##
        
        ## grab the states at time t - 1
        currStates = r[ t - 1, 5, : ]

        ## for a in ___agents____
        for a in 1 : agents
           
            ## excitation information
            excitation = strengthExcitation * sum( D[ a, findall( states .== 1 ) ] )
            
            ## inhibition information
            inhibition = strengthInhibition * sum( D[ a, findall( states .== 0 ) ] )
            
            ## decision making function
            decision = excitation - inhibition
            
            ## if we pass threshold
            if decision > 1
                
                ## update state
                r[ t, 5, a ] = 1
                
            end
            
        end
        
        ## -
        ## trigger false alarm
        ## -
        
        ## initiate
        if t == trigger

            ## switch state
            r[ t, 5, sample( 1 : agents, 1 ) ] .= 1

        end

    end
    
    ## return results
    return r
    
    end;

In [254]:
## run my simulation once
r = scaredyFish()

20×6×15 Array{Float64,3}:
[:, :, 1] =
  1.0  1.0  1.72179   3.03333   0.0  1.0
  2.0  1.0  1.80654   3.05283   0.0  1.0
  3.0  1.0  1.77326   3.00772   0.0  1.0
  4.0  1.0  1.64088   2.95984   0.0  1.0
  5.0  1.0  1.60308   2.96203   0.0  1.0
  6.0  1.0  1.63541   3.01602   0.0  1.0
  7.0  1.0  1.62679   3.0115    0.0  1.0
  8.0  1.0  1.5556    3.063     0.0  1.0
  9.0  1.0  1.56042   3.0271    0.0  1.0
 10.0  1.0  1.49389   2.99912   0.0  1.0
 11.0  1.0  1.48842   3.01157   1.0  0.0
 12.0  1.0  1.25409   2.72038   1.0  0.0
 13.0  1.0  0.0       0.0       0.0  0.0
 14.0  1.0  0.365795  0.352104  0.0  0.0
 15.0  1.0  0.59783   0.766725  0.0  0.0
 16.0  1.0  0.595068  0.801051  0.0  0.0
 17.0  1.0  0.634592  0.715507  0.0  0.0
 18.0  1.0  0.630637  0.705806  0.0  0.0
 19.0  1.0  0.589509  0.705357  0.0  0.0
 20.0  1.0  0.570798  0.667151  0.0  0.0

[:, :, 2] =
  1.0  2.0  1.83417  3.90539  0.0  1.0
  2.0  2.0  1.89643  3.8415   0.0  1.0
  3.0  2.0  1.8297   3.85886  0.0  1.0
  4.0  2.0  

In [251]:
## extract current coordinates
C = hcat( r[ 5, 3, : ], r[ 5, 4, : ] )

## grab states at this time
states = r[ 5, 5, : ]

## calculate distances between agents
D = pairwise( Euclidean(), C; dims = 1 )

## threshold this matrix
D[ D .> sensory ] .= 0

## convert to 1 / D ^ 2
D[ D .> 0 ] = 1 ./ D[ D .> 0 ] .^ 2

## grab a row
print( D[ 1, findall( states .== 0 ) ] )

## excitation information
excitation = 0.5 * sum( D[ a, findall( states .== 1 ) ] )

## inhibition information
inhibition = 0.01 * sum( D[ a, findall( states .== 0 ) ] )

## decision making function
[excitation - inhibition]

[0.0, 0.4252888059221655, 7.532323157225774, 0.3239544900853387, 0.41898829261852777, 0.42764657331288897, 15.050596689654457, 2.218449958237234, 0.42805222500364287, 113.40430145527047, 0.4406479672478834, 0.4640248273607772, 0.44108788677739436, 0.4925649749008904]

1-element Array{Float64,1}:
 -8.775966198015304

In [203]:
size( r )

(10, 6, 10)